# Algorithm: Interior-Point Methods
Interior-point methods solve a linear program by traversing the _interior_ of the feasible region—avoiding the corners until the very end—using a _barrier function_ to enforce the inequalities. They trace a path defined by modified KKT conditions (with logarithmic penalties) and converge to the optimum in a small, predictable number of steps.

> __The big idea__: Interior-point methods navigate the feasible region's interior using a smooth barrier that dominates at edges, taking Newton-style steps along a central path to the optimum.
> * __Barrier function?__ An invisible wall that shoots to infinity as you approach any constraint boundary, repelling your iterates and keeping them safely in the interior while still guiding you toward the optimum. Gradually lowering its weight lets the solution creep closer to the boundary without crossing it.
> * __Logarithmic penalties?__ An example barrier function. Logarithmic penalties are terms of the form $-\mu\ln(s)$ added to your objective. Because $\ln(s)\to-\infty$ as $s\to0^+$, they impose a _huge cost_ for getting too close to a constraint boundary. As you decrease the weight $\mu$, you gradually lessen the penalty's influence, letting the solution drift closer to the true feasible region edge.
> 
> Unlike the simplex method, which moves along the boundary and zig-zags across facets, these algorithms follow a direct, well-conditioned route through the interior, offering more predictable performance on large or ill-conditioned problems.

### Algorithm
Let's sketch out an interior-point algorithm to solve an LP.

__Initialization__: Given a linear program of the form: $\min\left\{c^\top x\mid Ax+s = b,\;x\ge0,\; s\ge0,\;b\ge0\right\}$ where $A\in\mathbb{R}^{m\times{n}}$. Specify an _initial strictly feasible guess_ for $(x^{(0)}, s^{(0)}, \lambda^{(0)}, \nu^{(0)})$ (more on this later). Specify a tolerance $\epsilon>0$, a maximum number of iterations $T$, an iteration counter $t\gets{0}$, set $\texttt{converged}\gets\texttt{false}$, and choose a _reduction factor_ $\sigma\in(0,1)$. Finally, compute $\mu^{(0)}$:
$$
\begin{align*}
\mu^{(0)} \gets \frac{x^{(0)^{\top}}\nu^{(0)} + s^{(0)^{\top}}\lambda^{(0)}}{n+m}
\end{align*}
$$

While not $\texttt{converged}$ __do__:
1. Compute the four residuals $(r_{P}, r_{D}, r^{x}_{C}, r^{s}_{C})$:
     - _Primal residual_: $r_{P}\gets{Ax^{(t)} +s^{(t)} - b}$. The primal residual $r_{P}$ shows how much your guess $(x,s)$ violates the constraints. Each entry of $r_P$ is the gap between your decision variables, slack variables, and the right-hand side $b$; zero indicates your solution is feasible.
     - _Dual residual_: $r_D \gets A^{\top}\lambda^{(t)}+\nu^{(t)} - c$. The dual residual $r_{D}$ indicates how much your dual variables $(\lambda,\nu)$ deviate from the stationarity condition of the Lagrangian. A nonzero $r_D$ shows you need to adjust the multipliers or primal $x$ to move toward optimality.
     - _Complementarity residual_: $r^{x}_{C}\gets X^{(t)}\nu^{(t)} - \mu^{(t)}\mathbf{1}$, where $X = \text{diag}(x)$. The complementarity residual $r^{x}_{C}$ measures deviation from the ideal central product $x_j \nu_j = \mu$ for each coordinate. When $r_C^x=0$, each $x_j$ and its dual partner $\nu_j$ satisfy the perturbed complementary-slackness condition, placing you on the central path between primal and dual feasibility.
     - _Slack-complementarity residual_: $r^{s}_{C} \gets S^{(t)}\lambda^{(t)} - \mu^{(t)}\mathbf{1}$, where $S=\text{diag}(s)$. The slack-complementarity residual $r^{s}_{C}$ measures how far each slack variable $s_i$ and its dual multiplier $\lambda_i$ are from satisfying the central-path condition. When $r_C^s = 0$, each slack $s_{i}$ and multiplier $\lambda_{i}$ satisfy $s_{i}\lambda_{i} = \mu$.

2. Compute the Jacobian matrix $J$:
$$
     J =
     \begin{pmatrix}
       A & I & 0 & 0\\
       0 & 0 & A^\top & I\\
       \text{diag}(\nu^{(t)}) & 0 & 0 & \text{diag}(x^{(t)})\\
       0 & \text{diag}(\lambda^{(t)}) & \text{diag}(s^{(t)}) & 0
     \end{pmatrix}.
   $$
3. Take a _Newton step_: solve for the updates $\Delta x, \Delta s, \Delta \lambda, \Delta \nu$:
$$
     J\,
     \begin{pmatrix}
       \Delta x\\[3pt]\Delta s\\[3pt]\Delta\lambda\\[3pt]\Delta\nu
     \end{pmatrix}
     = -\,
     \begin{pmatrix}
       r_P\\r_D\\r_C^x\\r_C^s
     \end{pmatrix}.
   $$

4. Choose the step size $\alpha$: Choose the largest $\alpha\in(0,1]$ such that:
   $$
     x^{(t)} + \alpha\,\Delta x > 0,\quad
     s^{(t)} + \alpha\,\Delta s > 0,\quad
     \lambda^{(t)} + \alpha\,\Delta\lambda > 0,\quad
     \nu^{(t)} + \alpha\,\Delta\nu > 0.
   $$
5. Update the system solution $(x,s,\lambda,\nu,\mu)$:
 - Set $x^{(t+1)} \gets x^{(t)} + \alpha\,\Delta x$.
 - Set $s^{(t+1)} \gets s^{(t)} + \alpha\,\Delta s$.
 - Set $\lambda^{(t+1)} \gets \lambda^{(t)} + \alpha\,\Delta\lambda$.
 - Set $\nu^{(t+1)} \gets \nu^{(t)} + \alpha\,\Delta\nu$.
 - Set $\mu^{(t+1)} \gets \sigma\,\mu^{(t)}$.

6. Check for convergence. Update the iteration counter $t\gets{t+1}$.
   - If $\mu^{(t)} \leq \epsilon$ (or $\|F\| \leq \epsilon$), then $(x^{(t)},s^{(t)})$ approximates the true optimum. Set $\texttt{converged}\gets\texttt{true}$ and return $(x^{(t)},s^{(t)})$. Here, $F$ is the stacked residual vector.
   - If $t>T$, we've run out of iterations. Set $\texttt{converged}\gets\texttt{true}$ and return $(x^{(t)},s^{(t)})$, which is the best solution we have so far.

**Relation to KKT**: At each $\mu>0$, the algorithm solves the **perturbed** KKT system
$\;r_P=0,\;r_D=0,\;r_C^x=0,\;r_C^s=0$,
which approaches the **exact** (unperturbed) KKT conditions as $\mu\to0$.

___